In [1]:
import sagemaker
import boto3
import pandas as pd
from sagemaker import get_execution_role

In [2]:
# get region name
region = boto3.Session().region_name
print ('region -> {}'.format(region))

region -> eu-west-1


In [3]:
# initialize session
session = sagemaker.Session()

# bucket details
bucket = 'snowflake-getting-started'
prefix = 'bank-marketing'

# get execution role
role = get_execution_role()

In [4]:
#download the dataset - this could be coming from snowflake as well
!wget -N https://datahub.io/machine-learning/bank-marketing/r/bank-marketing.csv
local_data_path = 'bank-marketing.csv'

--2020-07-25 10:22:56--  https://datahub.io/machine-learning/bank-marketing/r/bank-marketing.csv
Resolving datahub.io (datahub.io)... 172.67.157.38, 104.18.49.253, 104.18.48.253, ...
Connecting to datahub.io (datahub.io)|172.67.157.38|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://pkgstore.datahub.io/machine-learning/bank-marketing/bank-marketing_csv/data/000466faca88d1fc9961b0fe85ff995f/bank-marketing_csv.csv [following]
--2020-07-25 10:22:57--  https://pkgstore.datahub.io/machine-learning/bank-marketing/bank-marketing_csv/data/000466faca88d1fc9961b0fe85ff995f/bank-marketing_csv.csv
Resolving pkgstore.datahub.io (pkgstore.datahub.io)... 172.67.157.38, 104.18.48.253, 104.18.49.253, ...
Connecting to pkgstore.datahub.io (pkgstore.datahub.io)|172.67.157.38|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3700750 (3.5M) [text/csv]
Saving to: ‘bank-marketing.csv’

bank-marketing.csv  100%[===================>]   3.53M  --.-KB

In [5]:
!ls -lrt

total 3892
-rw-r--r--  1 root root 3700750 Aug 22  2018 bank-marketing.csv
drwxr-xr-x 25 root root    6144 Jul 21 11:04 amazon-sagemaker-examples
-rw-r--r--  1 root root  193631 Jul 22 11:55 SageMaker-Enable-Model-Monitor.ipynb
drwxr-xr-x  8 root root    6144 Jul 24 19:59 aws_sagemaker
drwxr-xr-x  5 root root    6144 Jul 24 20:05 ml-mushrooms-data-classifiers
-rw-r--r--  1 root root   20496 Jul 25 06:47 SageMakerAutopilotDataExplorationNotebook.ipynb
-rw-r--r--  1 root root   47921 Jul 25 06:47 SageMakerAutopilotCandidateDefinitionNotebook.ipynb


In [6]:
RANDOM_STATE = 99

# load csv in memory
data = pd.read_csv('bank-marketing.csv')
data.head()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,Class
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,1
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,1
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,1
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,1
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,1


In [9]:
# split data into test and training
train_data = data.sample(frac=0.8, random_state=RANDOM_STATE)
test_data = data.drop(train_data.index)

test_data = test_data.drop(['Class'],axis=1)
test_data.head()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16
13,58,technician,married,unknown,no,71,yes,no,unknown,5,may,71,1,-1,0,unknown
14,57,services,married,secondary,no,162,yes,no,unknown,5,may,174,1,-1,0,unknown
18,60,retired,married,primary,no,60,yes,no,unknown,5,may,219,1,-1,0,unknown
33,59,blue-collar,married,secondary,no,0,yes,no,unknown,5,may,226,1,-1,0,unknown
36,25,blue-collar,married,secondary,no,-7,yes,no,unknown,5,may,365,1,-1,0,unknown


In [10]:
# put train data into csv
train_data.to_csv('train_data.csv', index=False, header=True)

# upload train csv to s3
train_data_s3_path = session.upload_data(path='train_data.csv',
                                         bucket=bucket,
                                        key_prefix=prefix+'/train')
print('train data uploaded to {}'.format(train_data_s3_path))

train data uploaded to s3://snowflake-getting-started/bank-marketing/train/train_data.csv


In [11]:
# put test data into csv
test_data.to_csv('test_data.csv', index=False, header=True)

# upload test csv to s3
test_data_s3_path = session.upload_data(path='test_data.csv',
                                         bucket=bucket,
                                        key_prefix=prefix+'/test')
print('train data uploaded to {}'.format(test_data_s3_path))

train data uploaded to s3://snowflake-getting-started/bank-marketing/test/test_data.csv
